# Template

This notebook allows users to find common and specific genes in their experiment of interest using an *existing* VAE model and selecting a template experiment that is *not* included in the training compendium.

This notebook will generate a `generic_gene_summary_<experiment id>.tsv` file that contains z-scores per gene that indicates how specific a gene is the experiment in question.

In [1]:
%load_ext autoreload
%load_ext rpy2.ipython
%autoreload 2

In [2]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from ponyo import utils, simulate_expression_data
from sophie import (
    process,
    new_experiment_process, #REMOVE
    stats,
    ranking,
)
# Remove later
import pickle

/home/alexandra/anaconda3/envs/sophie/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
Using TensorFlow backend.


## Inputs

User needs to fill in the [config file](config_new_experiment.tsv) following the instructions provided in the [readme]](README.md)

In [3]:
# Read in config variables
base_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))

config_filename = "config_example.tsv"

params = utils.read_config(config_filename)

In [4]:
# Load config params

# Local directory to store intermediate files
local_dir = params["local_dir"]

# File containing un-normalized template experiment
raw_template_filename = params["raw_template_filename"]

# Un-normalized compendium filename
raw_compendium_filename = params["raw_compendium_filename"]

# Normalized compendium filename
normalized_compendium_filename = params["normalized_compendium_filename"]

# ID for template experiment to be selected
project_id = params["project_id"]

# Number of simulated experiments to generate
num_runs = params["num_simulated"]

# Directory containing trained VAE model
vae_model_dir = params["vae_model_dir"]

# Size of the latent dimension
latent_dim = params["latent_dim"]

# Scaler transform used to scale compendium data into 0-1 range for training
scaler_filename = params["scaler_filename"]

# Which DE method to use
# We recommend that if data is RNA-seq then use DESeq2
# If data is microarray then use Limma
de_method = params["DE_method"]

# If using DE-seq, setting this parameter will
# remove genes below a certain threshold
count_threshold = params["count_threshold"]

# Metadata file that specifies which samples to keep for DE analysis (Optional)
template_process_samples_filename = params["template_process_samples_filename"]
    
# Metadata file that specifies sample grouping for DE analysis
template_DE_grouping_filename = params["template_DE_grouping_filename"]

# Statistic to use to rank genes or pathways by
# Choices are {} FILL IN
col_to_rank_genes = params["rank_genes_by"]

In [5]:
# Files generated by this notebook

# File storing template experiment with gene ids mapped to compendium gene ids
mapped_template_filename = params["mapped_template_filename"]

# File storing normalized template experiment
normalized_template_filename = params["normalized_template_filename"]

# File storing processed template experiment,
# after samples have been selected for comparison in DE analysis
processed_template_filename = params["processed_template_filename"]

# Output summary file
output_filename = params["output_filename"]

## Normalize template experiment

Use scaler transform that was used to normalize the training compendium

In [6]:
#test = pd.read_csv(processed_template_filename, sep="\t", index_col=0, header=0)

In [7]:
"""print(test.shape)
test.head()"""

'print(test.shape)\ntest.head()'

In [8]:
"""# Load pickled file
scaler = pickle.load(open(scaler_filename, "rb"))"""

'# Load pickled file\nscaler = pickle.load(open(scaler_filename, "rb"))'

In [9]:
"""# Read data
raw_template = pd.read_csv(
    raw_template_filename, header=0, sep="\t", index_col=0
)

# Normalize the data using scaler transform in order to simulate data
normalized_template = scaler.transform(raw_template)

normalized_template_df = pd.DataFrame(
    normalized_template,
    columns=raw_template.columns,
    index=raw_template.index,
)

# Save normalized template
normalized_template_normalized.to_csv(normalized_template_filename, sep="\t")"""

'# Read data\nraw_template = pd.read_csv(\n    raw_template_filename, header=0, sep="\t", index_col=0\n)\n\n# Normalize the data using scaler transform in order to simulate data\nnormalized_template = scaler.transform(raw_template)\n\nnormalized_template_df = pd.DataFrame(\n    normalized_template,\n    columns=raw_template.columns,\n    index=raw_template.index,\n)\n\n# Save normalized template\nnormalized_template_normalized.to_csv(normalized_template_filename, sep="\t")'

In [10]:
"""mapped_compendium_filename = "/home/alexandra/Documents/Data/Generic_expression_patterns/mapped_recount2_compendium.tsv"
normalized_compendium = pd.read_csv(mapped_compendium_filename, sep="\t", index_col=0, header=0)
print(normalized_compendium.shape)"""

'mapped_compendium_filename = "/home/alexandra/Documents/Data/Generic_expression_patterns/mapped_recount2_compendium.tsv"\nnormalized_compendium = pd.read_csv(mapped_compendium_filename, sep="\t", index_col=0, header=0)\nprint(normalized_compendium.shape)'

In [11]:
#normalized_compendium.head()

In [12]:
# REad in raw_template
#raw_template_filename = "/home/alexandra/Documents/Data/Generic_expression_patterns/Costello_BladderCancer_example.txt"
#raw_template = pd.read_csv(raw_template_filename, sep="\t", index_col=0, header=0)
#print(raw_template.shape)

In [13]:
#raw_template.T.head()

In [14]:
#raw_template.T.to_csv("../data/input/template_data_example.tsv", sep="\t")

## Process template experiment

This step:
1. Normalizes the template experiment such that the template experiment and compendium experiment are in the same range
2. Ensures that the feature space (i.e. gene ids) are the same in the template and compendium

In [15]:
new_experiment_process.process_template_experiment(
    raw_template_filename,
    raw_compendium_filename,
    scaler_filename,
    mapped_template_filename,
    normalized_template_filename,
)

(72, 58528)
(49651, 17755)


## Simulate data

In [16]:
# Simulate multiple experiments UPDATE COMMENT
# This step creates the following files in "<local_dir>/pseudo_experiment/" directory:
#   - selected_simulated_data_SRP012656_<n>.txt
#   - selected_simulated_encoded_data_SRP012656_<n>.txt
#   - template_normalized_data_SRP012656_test.txt
# in which "<n>" is an integer in the range of [0, num_runs-1]

# REMOVE LATER
dataset_name = "pre_model_unseen_template"
# Load pickled file
scaler = pickle.load(open(scaler_filename, "rb"))

# Update simulated dir
os.makedirs(os.path.join(local_dir, "pseudo_experiment"), exist_ok=True)

# Update to take in file to be consisten
normalized_compendium = pd.read_csv(
    normalized_compendium_filename, header=0, sep="\t", index_col=0
)
normalized_template = pd.read_csv(
    normalized_template_filename, header=0, sep="\t", index_col=0
)
#------------
# Update call when new version of ponyo
for run_id in range(num_runs):
    new_experiment_process.embed_shift_template_experiment(
        normalized_compendium,
        normalized_template,
        vae_model_dir,
        project_id,
        scaler_filename,
        local_dir,
        latent_dim,
        run_id,
    )

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [18]:
template_process_samples_filename

'costello_process_samples.tsv'

## Process template and simulated experiments

* Remove samples not required for comparison
* Make sure ordering of samples matches metadata for proper comparison
* Make sure values are cast as integers if using DESeq
* Filter lowly expressed genes if using DESeq

In [19]:
## Update simulated dir
if not os.path.exists(template_process_samples_filename):
    template_process_samples_filename = None

if de_method == "deseq":
    # Process template data
    stats.process_samples_for_DESeq(
        raw_template_filename,
        template_DE_grouping_filename,
        processed_template_filename,
        count_threshold,
        template_process_samples_filename,
    )

    # Process simulated data
    for i in range(num_runs):
        simulated_filename = os.path.join(
            local_dir,
            "pseudo_experiment",
            f"selected_simulated_data_{project_id}_{i}.txt",
        )
        out_simulated_filename = os.path.join(
            local_dir,
            "pseudo_experiment",
            f"selected_simulated_data_{project_id}_{i}_processed.txt",
        )
        stats.process_samples_for_DESeq(
            simulated_filename,
            template_DE_grouping_filename,
            out_simulated_filename,
            count_threshold,
            template_process_samples_filename,
        )
else:
    stats.process_samples_for_limma(
        raw_template_filename,
        template_DE_grouping_filename,
        processed_template_filename,
        template_process_samples_filename,
    )

    for i in range(num_runs):
        simulated_filename = os.path.join(
            local_dir,
            "pseudo_experiment",
            f"selected_simulated_data_{project_id}_{i}.txt",
        )
        stats.process_samples_for_limma(
            simulated_filename,
            template_DE_grouping_filename,
            None,
            template_process_samples_filename,
        )

sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly


## Differential expression analysis

In [20]:
# Create subdirectory: "<local_dir>/DE_stats/"
os.makedirs(os.path.join(local_dir, "DE_stats"), exist_ok=True)

In [21]:
# Pass simulated dir to R scripts below

In [22]:
%%R -i template_DE_grouping_filename -i project_id -i processed_template_filename -i local_dir -i base_dir -i de_method

source(paste0(base_dir, '/sophie/DE_analysis.R'))

# File created: "<local_dir>/DE_stats/DE_stats_template_data_<project_id>_real.txt"
if (de_method == "deseq"){
    get_DE_stats_DESeq(
        template_DE_grouping_filename,
        project_id,
        processed_template_filename,
        "template",
        local_dir,
        "real"
    )
}
else{
    get_DE_stats_limma(
        template_DE_grouping_filename,
        project_id,
        processed_template_filename,
        "template",
        local_dir,
        "real"
    )
}

R[write to console]: Loading required package: S4Vectors

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following object is masked from ‘package:limma’:

    plotMA


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply

[1] "Checking sample ordering..."
[1] TRUE


In [23]:
%%R -i template_DE_grouping_filename -i project_id -i base_dir -i local_dir -i num_runs -i de_method

source(paste0(base_dir, '/sophie/DE_analysis.R'))

# Files created: "<local_dir>/DE_stats/DE_stats_simulated_data_<project_id>_<n>.txt"
for (i in 0:(num_runs-1)){
    simulated_data_filename <- paste(
        local_dir,
        "pseudo_experiment/selected_simulated_data_",
        project_id,
        "_",
        i,
        "_processed.txt",
        sep = ""
    )
    if (de_method == "deseq"){
        get_DE_stats_DESeq(
            template_DE_grouping_filename,
            project_id,
            simulated_data_filename,
            "simulated",
            local_dir,
            i
            )
    }
    else {
        get_DE_stats_limma(
            template_DE_grouping_filename,
            project_id,
            simulated_data_filename,
            "simulated",
            local_dir,
            i
            )
        }
    }

[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE


## Rank genes

Genes are ranked by their "generic-ness" - how frequently these genes are changed across the simulated experiments using user-specific test statistic provided in the `col_to_rank_genes` params (i.e. log2 fold change).

In [24]:
analysis_type = "DE"
template_DE_stats_filename = os.path.join(
    local_dir, "DE_stats", f"DE_stats_template_data_{project_id}_real.txt"
)

# Added
if de_method == "deseq":
    logFC_name = "log2FoldChange"
    pvalue_name = "padj"
else:
    logFC_name = "logFC"
    pvalue_name = "adj.P.Val"

template_DE_stats, simulated_DE_summary_stats = ranking.process_and_rank_genes_pathways(
    template_DE_stats_filename,
    local_dir,
    num_runs,
    project_id,
    analysis_type,
    col_to_rank_genes,
    logFC_name,
    pvalue_name,
)

/home/alexandra/anaconda3/envs/sophie/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


## Summary table

* Gene ID: Gene identifier (hgnc symbols for human data or PA number for *P. aeruginosa* data)
* (Real): Statistics for template experiment
* (Simulated): Statistics across simulated experiments
* Number of experiments: Number of simulated experiments
* Z-score: High z-score indicates that gene is more changed in template compared to the null set of simulated experiments (high z-score = highly specific to template experiment)
* Percentile (simulated): percentile rank of the median(abs(log fold change)). So its the median absolute change for that gene across the 25 simulated experiments that is then converted to a percentile rank from 0 - 100. Where a higher percentile indicates that the gene was highly changed frequently and would suggest that the gene is more commonly DE.
* Percent DE (simulated): the fraction of the simulated experiments in which that gene was found to be DE using (log fold change > 1 and adjusted p-value < 0.05). _Note:_ you may find that many genes have a 0 fraction. This is because there is some compression that happens when pushing data through the VAE so the variance of the simulated experiments is lower compared to the real experiment. We are aware of this limitation in the VAE and are looking at how to improve the variance and biological signal captured by the VAE, however we were still able to demonstrate that for now the VAE is able to simulate realistic looking biological experiments in our previous [paper](https://academic.oup.com/gigascience/article/9/11/giaa117/5952607).


**Note:**
* If using DESeq, genes with NaN in only the `Adj P-value (Real)` column are those genes flagged because of the `cooksCutoff` parameter. The cook's distance as a diagnostic to tell if a single sample has a count which has a disproportionate impact on the log fold change and p-values. These genes are flagged with an NA in the pvalue and padj columns of the result table.

* If using DESeq with count threshold, some genes may not be present in all simulated experiments (i.e. the `Number of experiments (simulated)` will not equal the number of simulated experiments you specified in the beginning. This pre-filtering will lead to some genes found in few simulated experiments and so the background/null set for that gene is not robust. Thus, the user should sort by both z-score and number of experiments to identify specific expressed genes.

* If using DESeq without count threshold, some genes may still not be present in all simulated experiments (i.e. the `Number of experiments (simulated)`  will not equal the number of simulated experiments you specified in the beginning. If the gene is 0 expressed across all samples and thus automatically given an NA in `log fold change, adjusted p-value` columns. Thus, the user should sort by both z-score and number of experiments to identify specific expressed genes.

For more information you can read [DESeq FAQs](https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#pvaluesNA)

In [25]:
# Get summary table
summary_gene_ranks = ranking.generate_summary_table(
    template_DE_stats_filename,
    template_DE_stats,
    simulated_DE_summary_stats,
    col_to_rank_genes,
    local_dir,
    "gene",
    params,
)

summary_gene_ranks.sort_values(by="Z score", ascending=False).head(10)

,Gene ID,Adj P-value (Real),Rank (Real),abs(log2FoldChange) (Real),log2FoldChange (Real),Median adj p-value (simulated),Rank (simulated),Percentile (simulated),Percent DE (simulated),Mean abs(log2FoldChange) (simulated),Std deviation (simulated),Number of experiments (simulated),Z score
PANK2,PANK2,0.525814,11433.0,0.379906,-0.379906,0.999997,8905.0,50.154903,0.0,0.259165,4.915525e-07,2,245632.964513
ZNF625,ZNF625,0.961887,12053.0,0.430418,-0.430418,0.999997,10411.0,58.637977,0.0,0.314954,5.671900e-07,2,203573.293727
TXLNB,TXLNB,0.734344,16593.0,2.150515,2.150515,0.999997,10887.0,61.319214,0.0,0.335735,1.022093e-04,2,17755.534033
PDE12,PDE12,0.568180,9507.0,0.272788,0.272788,0.999997,336.0,1.887005,0.0,0.034918,3.460411e-05,2,6874.040517
ANKRD26P3,ANKRD26P3,0.892191,16051.0,1.495263,-1.495263,0.999997,10935.0,61.589590,0.0,0.337966,2.094056e-04,2,5526.580788
GLT1D1,GLT1D1,0.914081,16185.0,1.617061,1.617061,0.999997,172.0,0.963217,0.0,0.024775,5.671583e-04,2,2807.480689
ANK2,ANK2,0.870070,13933.0,0.667308,-0.667308,0.999997,6732.0,37.914719,0.0,0.196523,1.717210e-04,2,2741.567161
CLEC14A,CLEC14A,0.728431,16783.0,2.584066,2.584066,0.999997,12438.0,70.055765,0.0,0.427270,1.074722e-03,2,2006.840610
OLFML2B,OLFML2B,0.806599,14439.0,0.794325,0.794325,0.999997,11698.0,65.887456,0.0,0.379492,2.232117e-04,2,1858.474665
CCDC178,CCDC178,0.589087,16640.0,2.262789,-2.262789,0.999997,733.0,4.123247,0.0,0.051120,1.217958e-03,2,1815.883106


In [26]:
summary_gene_ranks.isna().any()

Gene ID                                 False
Adj P-value (Real)                       True
Rank (Real)                              True
abs(log2FoldChange) (Real)               True
log2FoldChange (Real)                    True
Median adj p-value (simulated)           True
Rank (simulated)                         True
Percentile (simulated)                   True
Percent DE (simulated)                  False
Mean abs(log2FoldChange) (simulated)     True
Std deviation (simulated)                True
Number of experiments (simulated)       False
Z score                                  True
dtype: bool

In [27]:
summary_gene_ranks[summary_gene_ranks.isna().any(axis=1)]

,Gene ID,Adj P-value (Real),Rank (Real),abs(log2FoldChange) (Real),log2FoldChange (Real),Median adj p-value (simulated),Rank (simulated),Percentile (simulated),Percent DE (simulated),Mean abs(log2FoldChange) (simulated),Std deviation (simulated),Number of experiments (simulated),Z score
FOXR2,FOXR2,0.396900,17150.0,5.767623,-5.767623,0.999995,411.0,2.309469,0.0,0.038196,NaN,1,NaN
LINC00317,LINC00317,0.717849,17046.0,3.732698,-3.732698,0.999995,17669.0,99.521208,0.0,1.929327,NaN,1,NaN
MMP26,MMP26,0.740303,17000.0,3.431926,3.431926,0.999995,212.0,1.188532,0.0,0.027622,NaN,1,NaN
LINC01049,LINC01049,0.804101,16775.0,2.557619,2.557619,0.999995,16299.0,91.804202,0.0,0.990380,NaN,1,NaN
KRT33B,KRT33B,0.327854,16684.0,2.338820,-2.338820,0.999995,9995.0,56.294711,0.0,0.297408,NaN,1,NaN
ATP6V1G3,ATP6V1G3,0.657878,16586.0,2.141813,2.141813,0.999995,8917.0,50.222498,0.0,0.259435,NaN,1,NaN
DEFB119,DEFB119,0.933340,16574.0,2.125025,2.125025,0.999995,17714.0,99.774686,0.0,2.338129,NaN,1,NaN
IL2,IL2,0.951137,16434.0,1.909142,-1.909142,0.999995,17650.0,99.414184,0.0,1.842568,NaN,1,NaN
FATE1,FATE1,0.607663,16330.0,1.776036,-1.776036,0.999995,13628.0,76.758858,0.0,0.531702,NaN,1,NaN
PPY,PPY,0.722652,16324.0,1.771006,-1.771006,0.999995,13880.0,78.178336,0.0,0.559928,NaN,1,NaN


In [28]:
# Save
summary_gene_ranks.to_csv(output_filename, sep="\t")